In [1]:
import torch
from torch.nn import functional as F


def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = -1,
    gamma: float = 2,
    reduction: str = "none",
) -> torch.Tensor:
    """
    Loss used in RetinaNet for dense detection: https://arxiv.org/abs/1708.02002.
    Args:
        inputs: A float tensor of arbitrary shape.
                The predictions for each example.
        targets: A float tensor with the same shape as inputs. Stores the binary
                 classification label for each element in inputs
                (0 for the negative class and 1 for the positive class).
        alpha: (optional) Weighting factor in range (0,1) to balance
                positive vs negative examples. Default = -1 (no weighting).
        gamma: Exponent of the modulating factor (1 - p_t) to
               balance easy vs hard examples.
        reduction: 'none' | 'mean' | 'sum'
                 'none': No reduction will be applied to the output.
                 'mean': The output will be averaged.
                 'sum': The output will be summed.
    Returns:
        Loss tensor with the reduction option applied.
    """
    p = torch.sigmoid(inputs)
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

In [2]:
# 随便初始化一个输入
inputs = torch.tensor([-2, 2, 1.5]) # inputs是网络输出，没有直接的概率意义
targets = torch.tensor([0.0, 1.0, 0.0]) # 每一分量表示是该类别的概率

In [3]:
sigmoid_focal_loss(inputs, targets) # 直接用facebook写好的进行调用

tensor([0.0018, 0.0018, 1.1373])

# 下面我们分别输出一下每一步的结果

In [4]:
# 用sigmoid而不是softmax来获取是每一类的概率
p = torch.sigmoid(inputs)
p

tensor([0.1192, 0.8808, 0.8176])

In [5]:
# 用二分类交叉熵来对每一个类别获取交叉熵损失
# 这里输出是一个长度3的向量，直接用多分类交叉熵这里会是一个值
ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
ce_loss

tensor([0.1269, 0.1269, 1.7014])

In [6]:
# p_t表示模型分类正确的概率
p_t = p * targets + (1 - p) * (1 - targets)
p_t

tensor([0.8808, 0.8808, 0.1824])

In [7]:
# 取最优的gamma，根据公式得到loss
gamma = 2
loss = ce_loss * ((1 - p_t) ** gamma)
loss

tensor([0.0018, 0.0018, 1.1373])

In [8]:
# 最优的alpha是0.25，这里根据alpha最初的设定来取值（>0.5来加强正类样本权重）
alpha = 0.75
alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
loss = alpha_t * loss
print(alpha_t)
print(loss)

tensor([0.2500, 0.7500, 0.2500])
tensor([0.0005, 0.0014, 0.2843])
